In [1]:
import tensorflow as tf
import numpy as np

def body(x):
    a = tf.random_uniform(shape=[2, 2], dtype=tf.int32, maxval=100)
    b = tf.constant(np.array([[1, 2], [3, 4]]), dtype=tf.int32)
    c = a + b
    return tf.nn.relu(x + c)

def condition(x):
    return tf.reduce_sum(x) < 1000

x = tf.Variable(tf.constant(0, shape=[2, 2]))

with tf.Session():
    tf.initialize_all_variables().run()
    result = tf.while_loop(condition, body, [x])
    print(result.eval())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[228 316]
 [251 258]]


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [3]:
import melt

tensorflow_version: 0.12.head


In [4]:
x = tf.constant([1,2])
melt.length(x, 0).eval()

2

In [5]:
 y = tf.constant([
                    [[1,2,3,4],
                     [2,2,3,4]],
                    [[1,4/16.0,5,7],
                     [0,0,0,0]],
                  ], dtype=tf.float32)

In [6]:
y.eval()

array([[[ 1.  ,  2.  ,  3.  ,  4.  ],
        [ 2.  ,  2.  ,  3.  ,  4.  ]],

       [[ 1.  ,  0.25,  5.  ,  7.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ]]], dtype=float32)

In [7]:
y.eval().shape

(2, 2, 4)

In [8]:
u = tf.nn.softmax(y[0])
v = tf.nn.softmax(y[1])
w = u + v

In [9]:
tf.shape(u).eval()

array([2, 4], dtype=int32)

In [10]:
w.eval()

array([[ 0.03423488,  0.08817232,  0.35570377,  1.52188897],
       [ 0.33259454,  0.33259454,  0.47451523,  0.86029565]], dtype=float32)

In [11]:
i = tf.constant(0)
c = lambda i: tf.less(i, melt.length(x,0))
b = lambda i: tf.add(i, 1)
r = tf.while_loop(c, b, [i])

In [12]:
r.eval()

2

In [13]:
i = tf.constant(0)
tf.nn.softmax(y[i, :, :]).eval()
z = tf.constant(0., shape=[2, 4], dtype=tf.float32)
z += tf.nn.softmax(y[i, :, :])
z.eval()

array([[ 0.0320586 ,  0.08714432,  0.23688281,  0.64391422],
       [ 0.08259454,  0.08259454,  0.22451523,  0.61029565]], dtype=float32)

In [14]:
i = tf.constant(0)

c = lambda i, z: tf.less(i, melt.length(x,0))
#b = lambda i: tf.add(i, 1）
z = tf.constant(0., shape=[2, 4], dtype=tf.float32)

def b(i, z):
    #-----below only ok in tf11.0  TODO
    #z += tf.nn.softmax(y[i, :, :])
    m = y[i, :, :]
    #m = tf.slice(y, [i, 0, 0], [1, -1, -1])
    z += tf.nn.softmax(m)
    return tf.add(i, 1), tf.reshape(z, [2,4])
r = tf.while_loop(c, b, [i, z])

In [15]:
r[0].eval()

2

In [16]:
r[1].eval()

array([[ 0.03423489,  0.08817232,  0.3557038 ,  1.52188897],
       [ 0.33259454,  0.33259454,  0.47451523,  0.86029565]], dtype=float32)

In [17]:
z.eval()

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]], dtype=float32)

In [18]:
class Worker:
    def __init__(self):
        self.i = tf.constant(0)
        self.z = tf.constant(0., shape=[2, 4], dtype=tf.float32)
    def step(self):
        self.z += 1
        return self.z
    
c = lambda i, z: tf.less(i, 2)

worker = Worker()
def step():
    global worker
    return worker.step()

def b(i, z):
    m = y[i, :, :]
    z += step() + tf.nn.softmax(m)
    return tf.add(i, 1), tf.reshape(z, [2,4])


r = tf.while_loop(c, b, [worker.i, worker.z])

In [19]:
r[0].eval()

2

In [20]:
r[1].eval()

array([[ 2.034235  ,  2.08817244,  2.35570383,  3.52188897],
       [ 2.33259439,  2.33259439,  2.4745152 ,  2.86029577]], dtype=float32)

In [21]:
worker.i.eval()

0

In [23]:
worker.z

<tf.Tensor 'while_3/add_1:0' shape=(2, 4) dtype=float32>

In [34]:
melt.length(x, 0).eval()

2

In [81]:
y[i + 1, :, :].eval()

array([[ 1.  ,  0.25,  5.  ,  7.  ],
       [ 2.  ,  2.  ,  0.  ,  0.  ]], dtype=float32)

In [85]:
tf.nn.softmax(y[i + 1, :, :]).eval()

array([[ 0.00217628,  0.001028  ,  0.11882096,  0.87797475],
       [ 0.44039851,  0.44039851,  0.05960146,  0.05960146]], dtype=float32)

In [83]:
tf.slice(y, [i + 1, 0, 0], [1, -1, -1]).eval()

array([[[ 1.  ,  0.25,  5.  ,  7.  ],
        [ 2.  ,  2.  ,  0.  ,  0.  ]]], dtype=float32)

In [87]:
tf.nn.softmax(tf.slice(y, [i + 1, 0, 0], [1, -1, -1])).eval()

array([[[ 0.00217628,  0.001028  ,  0.11882096,  0.87797475],
        [ 0.44039851,  0.44039851,  0.05960146,  0.05960146]]], dtype=float32)

In [90]:
tf.squeeze?

In [18]:
idx = tf.constant([1,3])
melt.dynamic_gather2d(y[i + 1, :, :], idx).eval()

array([ 0.25,  0.  ], dtype=float32)

In [33]:
tf.log(melt.dynamic_gather2d(y[i + 1, :, :], idx) + 1e-12).eval()

array([ -1.38629436, -27.6310215 ], dtype=float32)

In [34]:
tf.log(tf.maximum(melt.dynamic_gather2d(y[i + 1, :, :], idx), 1e-12)).eval()

array([ -1.38629436, -27.6310215 ], dtype=float32)

In [20]:
tf.reduce_sum(tf.log(melt.dynamic_gather2d(y[i + 1, :, :], idx) + 1e-12)).eval()

-29.017315

In [21]:
selected_probs = melt.dynamic_gather2d(y[i + 1, :, :], idx) 

In [22]:
selected_log_probs = tf.log(selected_probs + 1e-12)

In [23]:
score = tf.reduce_sum(selected_log_probs)

In [24]:
normalize_factor = tf.pow(melt.length(x, 0), 1)

In [25]:
normalize_factor.eval()

2

In [27]:
score /= tf.cast(normalize_factor, tf.float32)

In [28]:
score.eval()

-14.508657

In [37]:
tf.pow?

In [48]:
temp = tf.constant([[1,2,3], [4,5,6]])
tf.reshape(tf.slice(temp, [0, 0], [-1, 1]), [-1]).eval()

array([1, 4], dtype=int32)

In [91]:
i = tf.constant(0)
seqence_length = melt.length(x, 0)
idx = tf.constant([1,3])
c = lambda i, log_probs: tf.less(i, seqence_length)
#b = lambda i: tf.add(i, 1）
log_probs = tf.constant(0., shape=[2,], dtype=tf.float32)
def b(i, log_probs):
    #-----below only ok in tf11.0  TODO
    #z += tf.nn.softmax(y[i, :, :])
    #m = y[i, :, :]
    m = tf.squeeze(tf.slice(y, [i, 0, 0], [1, -1, -1]))
    probs = tf.nn.softmax(m)
    selected_probs = melt.dynamic_gather2d(probs, idx)
    selected_log_probs = tf.log(tf.maximum(selected_probs, 1e-12))
    log_probs += selected_log_probs
    return tf.add(i, 1), tf.reshape(log_probs, [2,])
_, log_probs = tf.while_loop(c, b, [i, log_probs])
print log_probs.eval()
normalize_factor = tf.cast(tf.pow(seqence_length, 1), tf.float32)
log_probs /= normalize_factor
log_probs = tf.exp(log_probs)
#log_probs /= normalize_factor
print log_probs.eval()

[-9.32032776 -3.31388712]
[ 0.00946491  0.19072102]


In [50]:
tf.nn.softmax(tf.constant([0, 0, 0], tf.float32)).eval()

array([ 0.33333334,  0.33333334,  0.33333334], dtype=float32)

In [80]:
temp = tf.constant([[0,0,0,0], [3,0,0,-2]], dtype=tf.float32)
print tf.nn.top_k(temp)[0].eval()
print tf.arg_max(temp, 1).eval()
print tf.reduce_max(temp, 1).eval()
temp = tf.constant([3,0,0,-2], dtype=tf.float32)
print tf.nn.top_k(temp)[0].eval()
print tf.reduce_max(temp).eval()

[[ 0.]
 [ 3.]]
[0 0]
[ 0.  3.]
[ 3.]
3.0


In [64]:
tf.sign(temp).eval()

array([[ 0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -1.]], dtype=float32)

In [60]:
tf.sign?

In [62]:
tf.abs(temp).eval()

array([[ 0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  1.]], dtype=float32)

In [67]:
tf.nn.top_k?

In [92]:
tf.shape(tf.range(0,3)).eval()

array([3], dtype=int32)